In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
import os
import cv2
import numpy as np
import random
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix
import glob
from PIL import Image

In [2]:
# [실험 모드 선택] 
# EXP_NAME = 'BASELINE'  # 1. 기본 RGB (과적합 확인용)
EXP_NAME = 'ROBUST'    # 2. RGB + Mixup + Strong Aug (강건성)
# EXP_NAME = 'FREQ'      # 3. Frequency + Mixup (아티팩트)

# [데이터 경로 설정]
# 학습용: 6만장 데이터 (폴더 안에 Real, Fake 혹은 0, 1 폴더 존재 가정)
TRAIN_DATA_ROOT = '../datasets/deepfake/Train' 

# 평가용: Wiki/Inpainting 등 Cropped 데이터 (학습에 안 쓴 것!)
TEST_DATA_ROOT = '../datasets/deepfake_60k_cropped_224px'

# TRAIN_DATA_ROOT, TEST_DATA_ROOT = TEST_DATA_ROOT, TRAIN_DATA_ROOT 

TRAIN_N_SAMPLES = 60000
TEST_N_SAMPLES = 10000

IMG_SIZE = 224
BATCH_SIZE = 64 # 메모리 터지면 32로
EPOCHS = 15
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 전처리 및 증강 로직

In [3]:
"""주파수 도메인 변환 (uint8 -> uint8)"""
# def get_freq_feature(img_np):
#     blur = cv2.GaussianBlur(img_np, (21, 21), 0)
#     high_pass = cv2.addWeighted(img_np, 4, blur, -4, 128)
#     fft_img = np.zeros_like(img_np, dtype=np.float32)
#     for c in range(3):
#         f = np.fft.fft2(high_pass[:, :, c])
#         fshift = np.fft.fftshift(f)
#         magnitude = 20 * np.log(np.abs(fshift) + 1e-8)
#         fft_img[:, :, c] = magnitude
#     return cv2.normalize(fft_img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

# [수정 후: 고주파 노이즈 맵(Residual)만 반환]
def get_freq_feature(img_np):
    """
    FFT 변환을 수행하지 않고, 공간 도메인의 고주파 잔차(Residual)만 추출합니다.
    CNN이 훨씬 학습하기 쉬우면서도, Content(얼굴)가 아닌 Artifact(노이즈)에 집중하게 합니다.
    """
    # 1. Grayscale 변환 (색상 정보 제거, 텍스처 집중)
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    
    # 2. 고주파 추출 (High Pass Filter)
    # 원본 - 블러 = 엣지와 노이즈만 남음
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    high_pass = cv2.addWeighted(gray, 1.5, blur, -0.5, 0) # 샤픈 필터 느낌으로 강화
    
    # 혹은 라플라시안 필터 (더 강력한 엣지 검출)
    # high_pass = cv2.Laplacian(gray, cv2.CV_16S, ksize=3)
    # high_pass = cv2.convertScaleAbs(high_pass)
    
    # 3. 차원을 3채널로 맞춰줌 (ResNet 입력 호환성 위해)
    high_pass_3c = cv2.cvtColor(high_pass, cv2.COLOR_GRAY2RGB)
    
    # 4. 히스토그램 평활화 (노이즈 대비 극대화 - 옵션)
    # high_pass_3c[:,:,0] = cv2.equalizeHist(high_pass_3c[:,:,0])
    # high_pass_3c[:,:,1] = cv2.equalizeHist(high_pass_3c[:,:,1])
    # high_pass_3c[:,:,2] = cv2.equalizeHist(high_pass_3c[:,:,2])

    return high_pass_3c

In [4]:
"""강력한 화질 저하 증강 (학습 시에만)"""
def strong_aug(img_np):
    if random.random() < 0.5: # JPEG Compression
        q = random.randint(30, 90)
        _, enc = cv2.imencode('.jpg', img_np, [int(cv2.IMWRITE_JPEG_QUALITY), q])
        img_np = cv2.imdecode(enc, 1)
        img_np = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)

    if random.random() < 0.3: # Blur
        img_np = cv2.GaussianBlur(img_np, (5, 5), 0)
        
    if random.random() < 0.3: # Noise
        noise = np.random.normal(0, 15, img_np.shape).astype(np.uint8)
        img_np = np.clip(img_np.astype(np.int16) + noise, 0, 255).astype(np.uint8)
        
    return img_np

In [5]:
"""mixup 기법 적용 (label이 [0, 1] 이면 [0.1, 0.9] 이런식으로 바꿈)"""
def mixup_data(x, y, alpha=1.0):
    if alpha > 0: lam = np.random.beta(alpha, alpha)
    else: lam = 1
    index = torch.randperm(x.size(0)).to(DEVICE)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

## data load

In [6]:
def find_class_dirs(root_dir):

    print(f"Scanning: {os.path.abspath(root_dir)}") # 절대 경로 확인용 출력
    
    if not os.path.exists(root_dir):
        print(f"[Error] 경로가 존재하지 않습니다: {root_dir}")
        print("  -> 현재 작업 경로(os.getcwd()):", os.getcwd())
        return None, None

    real_dir, fake_dir = None, None
    
    # 1. 루트 바로 아래 탐색
    subdirs = [os.path.join(root_dir, d) for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
    
    # 2. 만약 바로 아래에 없다면 1 depth 더 들어가서 탐색 (예: Train/Train/Real...)
    if len(subdirs) == 1:
        nested_root = subdirs[0]
        print(f"  -> Checking nested dir: {nested_root}")
        subdirs = [os.path.join(nested_root, d) for d in os.listdir(nested_root) if os.path.isdir(os.path.join(nested_root, d))]

    for d in subdirs:
        name = os.path.basename(d).lower()
        if 'real' in name: real_dir = d
        elif 'fake' in name: fake_dir = d
            
    return real_dir, fake_dir

In [7]:
"""데이터 전체를 RAM에 로드하는 함수"""
def load_data_to_ram(root_dir, target_size=224, n_samples=None):
    print(f"Loading data from {root_dir} into RAM...")
    
    real_dir, fake_dir = find_class_dirs(root_dir)
    if not real_dir or not fake_dir:
        print(f"[Error] '{root_dir}'에서 Real/Fake 폴더를 못 찾았습니다.")
        return None, None

    print(f" - Real: {os.path.basename(real_dir)}")
    print(f" - Fake: {os.path.basename(fake_dir)}")

    # 1. 모든 경로 수집
    all_files = []
    for label, folder in enumerate([real_dir, fake_dir]): # 0: Real, 1: Fake
        files = glob.glob(os.path.join(folder, "*.*"))
        # 이미지 확장자 필터링
        files = [f for f in files if f.lower().endswith(('.jpg', '.png', '.jpeg', '.webp'))]
        for f in files:
            all_files.append((f, label))
            
    total_files = len(all_files)
    print(f" - Total Images Found: {total_files}")
    
    # 2. 샘플링 (n_samples 설정 시)
    if n_samples is not None and n_samples < total_files:
        print(f" - Sampling {n_samples} images...")
        random.shuffle(all_files)
        all_files = all_files[:n_samples]
    else:
        # 셔플은 학습 데이터 분포를 섞어주므로 기본적으로 해두는 게 좋음
        random.shuffle(all_files)

    # 3. 로딩 시작
    crop_transform = transforms.CenterCrop(target_size)
    resize_transform = transforms.Resize((target_size, target_size))
    to_uint8_tensor = transforms.PILToTensor() 

    error_count = 0 # 에러 카운트 추가
    
    tensor_list = []
    label_list = []
    
    for fpath, label in tqdm(all_files, desc="Loading to RAM", leave=False):
        try:
            img = Image.open(fpath).convert('RGB')
            w, h = img.size
            
            # 224보다 크거나 같으면 CenterCrop (화질 보존)
            if w >= target_size and h >= target_size:
                img = crop_transform(img)
            else:
                # 작으면 Resize
                img = resize_transform(img)
            
            # uint8 Tensor 변환
            t_img = to_uint8_tensor(img)
            
            tensor_list.append(t_img)
            label_list.append(label)
        except Exception as e:
             # [디버깅] 처음 10개 에러만 출력
            if error_count < 10:
                print(f"[Error] Failed to load {fpath}: {e}")
                error_count += 1
            pass 


    if not tensor_list:
        print("[Error] 이미지가 없습니다.")
        return None, None

    # Stack
    data_tensor = torch.stack(tensor_list)
    label_tensor = torch.tensor(label_list, dtype=torch.long)
    
    mem_size = data_tensor.element_size() * data_tensor.nelement() / (1024**3)
    print(f"Load Complete! {len(data_tensor)} images. RAM: {mem_size:.2f} GB")
    
    return data_tensor, label_tensor


In [8]:
"""dataset 정의"""
class RAMDataset(Dataset):
    def __init__(self, data_tensor, label_tensor, exp_name, is_train=True):
        self.data = data_tensor   
        self.labels = label_tensor
        self.exp_name = exp_name
        self.is_train = is_train
        self.normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        self.to_float = transforms.ToTensor() 

    def __len__(self): return len(self.data)

    def __getitem__(self, idx):
        img_tensor = self.data[idx]
        label = self.labels[idx]
        
        # Tensor -> Numpy (H, W, 3) for OpenCV
        img_np = img_tensor.permute(1, 2, 0).numpy()
        
        # Augmentation (Train)
        if self.is_train and self.exp_name in ['ROBUST', 'FREQ']:
            img_np = strong_aug(img_np)
        
        # Freq Conversion
        if self.exp_name == 'FREQ':
            img_np = get_freq_feature(img_np)
            
        # Numpy -> Float Tensor & Normalize
        img_out = self.to_float(img_np)
        img_out = self.normalize(img_out)
        
        return img_out, label

## train

In [9]:
print(f"🚀 Experiment: [{EXP_NAME}] (Target Size: {IMG_SIZE})")

# 1. Load Data
print(">>> Loading Train Set...")
train_x, train_y = load_data_to_ram(TRAIN_DATA_ROOT, IMG_SIZE, TRAIN_N_SAMPLES)
print(">>> Loading Test Set...")
test_x, test_y = load_data_to_ram(TEST_DATA_ROOT, IMG_SIZE, TEST_N_SAMPLES)

if train_x is None or test_x is None: 
    raise Exception("데이터 로드 실패. 경로를 확인하세요.")

# 2. Loader (num_workers=0 for RAM speed)
train_ds = RAMDataset(train_x, train_y, EXP_NAME, is_train=True)
test_ds = RAMDataset(test_x, test_y, EXP_NAME, is_train=False)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

🚀 Experiment: [ROBUST] (Target Size: 224)
>>> Loading Train Set...
Loading data from ../datasets/deepfake/Train into RAM...
Scanning: /home/dongwook/workspace/datasets/deepfake/Train
 - Real: Real
 - Fake: Fake
 - Total Images Found: 140002
 - Sampling 60000 images...


Load Complete! 60000 images. RAM: 8.41 GB
>>> Loading Test Set...
Loading data from ../datasets/deepfake_60k_cropped_224px into RAM...
Scanning: /home/dongwook/workspace/datasets/deepfake_60k_cropped_224px
 - Real: Real
 - Fake: Fake
 - Total Images Found: 59949
 - Sampling 10000 images...


Load Complete! 10000 images. RAM: 1.40 GB


In [ ]:
# 3. Model
if EXP_NAME == 'FREQ':
    model = models.resnet18(pretrained=False)
else:
    model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

best_acc = 0.0

In [ ]:
for epoch in range(EPOCHS):
    model.train()
    for inputs, labels in tqdm(train_loader, desc=f"Ep {epoch+1}", leave=False):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        
        if EXP_NAME != 'BASELINE':
            inputs, ta, tb, lam = mixup_data(inputs, labels, alpha=0.4)
            outputs = model(inputs)
            loss = mixup_criterion(criterion, outputs, ta, tb, lam)
        else:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    model.eval()
    preds, acts = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(DEVICE)
            outputs = model(inputs)
            preds.extend(torch.argmax(outputs, 1).cpu().numpy())
            acts.extend(labels.numpy())
            
    acc = accuracy_score(acts, preds)
    print(f"Epoch {epoch+1} | Test Acc: {acc:.4f}")
    
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), f"model_ram_{EXP_NAME}.pth")
        print("--> Best Model Saved!")

print(f"\n✅ Finished. Best Acc: {best_acc:.4f}")

In [ ]:
raise Exception

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_confusion_matrix(cm, acc):
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Pred: Real', 'Pred: Fake'], 
                yticklabels=['Act: Real', 'Act: Fake'])
    plt.title(f'Fusion Result (Acc: {acc:.4f})')
    plt.ylabel('Actual Label')
    plt.xlabel('Predicted Label')
    plt.show()

In [ ]:
def run_fusion():
    SWAP_DATASETS = False
    print("\n🚀 Running Model Fusion (Ensemble)...")
    
    # 1. Load Test Data (메모리에 한 번만 로드)
    # SWAP 여부에 따라 테스트 경로 자동 설정
    if 'SWAP_DATASETS' in globals() and SWAP_DATASETS:
        target_path = TRAIN_DATA_ROOT # Kaggle이 Test가 됨
        target_n = TRAIN_N_SAMPLES
        tag = "SWAP"
        print(f"   -> Mode: SWAP (Testing on Kaggle Data: {target_path})")
    else:
        target_path = TEST_DATA_ROOT # Wiki가 Test가 됨
        target_n = TEST_N_SAMPLES
        tag = "NORMAL"
        print(f"   -> Mode: NORMAL (Testing on Wiki Data: {target_path})")

    # RAM 로딩
    test_x, test_y = load_data_to_ram(target_path, IMG_SIZE, target_n)
    if test_x is None: 
        print("[Error] 테스트 데이터를 로드하지 못했습니다.")
        return

    # 2. Create Loaders (동일 데이터, 다른 전처리)
    # ROBUST용 로더
    ds_robust = RAMDataset(test_x, test_y, EXP_NAME='ROBUST', is_train=False)
    loader_robust = DataLoader(ds_robust, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
    # FREQ용 로더
    ds_freq = RAMDataset(test_x, test_y, EXP_NAME='FREQ', is_train=False)
    loader_freq = DataLoader(ds_freq, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
    # 3. Load Models
    model_robust = models.resnet18(pretrained=False) # 구조만 생성
    model_robust.fc = nn.Linear(512, 2)
    
    model_freq = models.resnet18(pretrained=False)
    model_freq.fc = nn.Linear(512, 2)
    
    # 모델 파일 찾기 (경로/이름 유연하게)
    path_robust = f"model_ram_ROBUST_{tag}.pth"
    if not os.path.exists(path_robust): path_robust = "model_ram_ROBUST.pth" # Fallback
    
    path_freq = f"model_ram_FREQ_{tag}.pth"
    if not os.path.exists(path_freq): path_freq = "model_ram_FREQ.pth" # Fallback
    
    print(f"   -> Loading ROBUST: {path_robust}")
    if os.path.exists(path_robust):
        model_robust.load_state_dict(torch.load(path_robust, map_location=DEVICE))
    else:
        print("[Warning] ROBUST 모델 파일이 없습니다! 앙상블 불가.")
        return

    print(f"   -> Loading FREQ:   {path_freq}")
    if os.path.exists(path_freq):
        model_freq.load_state_dict(torch.load(path_freq, map_location=DEVICE))
    else:
        print("[Warning] FREQ 모델 파일이 없습니다! 앙상블 불가.")
        return

    model_robust = model_robust.to(DEVICE).eval()
    model_freq = model_freq.to(DEVICE).eval()
    
    # 4. Inference loop
    all_preds = []
    all_labels = []
    
    print("   -> Starting Ensemble Inference...")
    with torch.no_grad():
        # zip으로 두 로더를 동시에 순회
        for (img_r, lbl), (img_f, _) in tqdm(zip(loader_robust, loader_freq), total=len(loader_robust)):
            img_r, img_f = img_r.to(DEVICE), img_f.to(DEVICE)
            
            # Softmax Probability 계산
            prob_r = torch.softmax(model_robust(img_r), dim=1)
            prob_f = torch.softmax(model_freq(img_f), dim=1)
            
            # Fusion (Soft Voting - 평균)
            final_prob = (prob_r + prob_f) / 2
            
            # 최종 예측
            preds = torch.argmax(final_prob, 1).cpu().numpy()
            
            all_preds.extend(preds)
            all_labels.extend(lbl.numpy())
            
    # 5. Analysis
    acc = accuracy_score(all_labels, all_preds)
    cm = confusion_matrix(all_labels, all_preds)
    
    print(f"\n✨ Final Fusion Accuracy: {acc:.4f}")
    
    # 혼동 행렬 텍스트 출력
    tn, fp, fn, tp = cm.ravel()
    print("-" * 40)
    print(f" Confusion Matrix:")
    print(f"               [Pred: Real] | [Pred: Fake]")
    print(f" [Act: Real]      {tn:^10} | {fp:^10}")
    print(f" [Act: Fake]      {fn:^10} | {tp:^10}")
    print("-" * 40)
    
    # 정밀도/재현율 (분석용 멘트 생성에 필요)
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    print(f" Precision (가짜라고 했을때 진짜 가짜인 비율): {precision:.4f}")
    print(f" Recall    (실제 가짜 중 찾아낸 비율)        : {recall:.4f}")
    
    # 그래프 그리기
    plot_confusion_matrix(cm, acc)

In [ ]:
run_fusion()

In [12]:
def find_best_threshold(use_fusion=False):
    mode_str = "Fusion Ensemble" if use_fusion else "Single Model (ROBUST)"
    print(f"\n🔍 Finding Best Threshold for [{mode_str}]...")
    
    # 1. Test Data Load
    test_x, test_y = load_data_to_ram(TEST_DATA_ROOT, IMG_SIZE, TEST_N_SAMPLES)
    if test_x is None: return

    all_probs = []
    all_labels = []

    # 2. Inference
    if use_fusion:
        print("   -> Loading ROBUST and FREQ models...")
        model_rgb = models.resnet18(weights=None); model_rgb.fc = nn.Linear(512, 2)
        model_freq = models.resnet18(weights=None); model_freq.fc = nn.Linear(512, 2)
        
        path_rgb, path_freq = "model_ram_ROBUST.pth", "model_ram_FREQ.pth"
        
        if os.path.exists(path_rgb) and os.path.exists(path_freq):
            model_rgb.load_state_dict(torch.load(path_rgb, map_location=DEVICE))
            model_freq.load_state_dict(torch.load(path_freq, map_location=DEVICE))
        else:
            print("[Error] 모델 파일이 없습니다. 먼저 학습을 완료하세요.")
            return

        model_rgb.to(DEVICE).eval()
        model_freq.to(DEVICE).eval()
        
        ds_rgb = RAMDataset(test_x, test_y, 'ROBUST', is_train=False)
        loader_rgb = DataLoader(ds_rgb, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        ds_freq = RAMDataset(test_x, test_y, 'FREQ', is_train=False)
        loader_freq = DataLoader(ds_freq, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        
        print("   -> Calculating Fusion Probabilities...")
        with torch.no_grad():
             for (img_r, lbl), (img_f, _) in tqdm(zip(loader_rgb, loader_freq), total=len(loader_rgb)):
                img_r, img_f = img_r.to(DEVICE), img_f.to(DEVICE)
                prob_r = torch.softmax(model_rgb(img_r), dim=1)
                prob_f = torch.softmax(model_freq(img_f), dim=1)
                avg_prob = (prob_r + prob_f) / 2
                all_probs.extend(avg_prob[:, 1].cpu().numpy())
                all_labels.extend(lbl.numpy())
    else:
        print("   -> Loading single ROBUST model...")
        model = models.resnet18(weights=None); model.fc = nn.Linear(512, 2)
        if os.path.exists("model_ram_ROBUST.pth"):
            model.load_state_dict(torch.load("model_ram_ROBUST.pth", map_location=DEVICE))
        else:
            print("[Error] model_ram_ROBUST.pth 파일이 없습니다.")
            return
        model.to(DEVICE).eval()
        
        ds = RAMDataset(test_x, test_y, 'ROBUST', is_train=False)
        loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        
        print("   -> Calculating Probabilities...")
        with torch.no_grad():
            for inputs, labels in tqdm(loader):
                inputs = inputs.to(DEVICE)
                probs = torch.softmax(model(inputs), dim=1)[:, 1]
                all_probs.extend(probs.cpu().numpy())
                all_labels.extend(labels.numpy())

    # 3. Threshold Search
    all_probs = np.array(all_probs)
    all_labels = np.array(all_labels)
    
    best_acc = 0
    best_thresh = 0.5
    
    print(f"\n[{mode_str}] Threshold Search Result:")
    for thresh in np.arange(0.1, 0.91, 0.01):
        preds = (all_probs >= thresh).astype(int)
        acc = accuracy_score(all_labels, preds)
        if acc > best_acc:
            best_acc = acc
            best_thresh = thresh
            
    print(f"\n✨ Best Accuracy: {best_acc:.4f} (Threshold: {best_thresh:.2f})")
    
    final_preds = (all_probs >= best_thresh).astype(int)
    cm = confusion_matrix(all_labels, final_preds)
    print("-" * 30)
    print(f" Confusion Matrix (Thresh {best_thresh:.2f}):")
    print(cm)
    print("-" * 30)
find_best_threshold(True)



🔍 Finding Best Threshold for [Fusion Ensemble]...
Loading data from ../datasets/deepfake_60k_cropped_224px into RAM...
Scanning: /home/dongwook/workspace/datasets/deepfake_60k_cropped_224px
 - Real: Real
 - Fake: Fake
 - Total Images Found: 59949
 - Sampling 10000 images...


Load Complete! 10000 images. RAM: 1.40 GB
   -> Loading ROBUST and FREQ models...
   -> Calculating Fusion Probabilities...


100%|██████████| 157/157 [00:09<00:00, 16.39it/s]


[Fusion Ensemble] Threshold Search Result:

✨ Best Accuracy: 0.5820 (Threshold: 0.31)
------------------------------
 Confusion Matrix (Thresh 0.31):
[[2373 2574]
 [1606 3447]]
------------------------------


In [10]:
def run_domain_adaptation():
    print("\n🚀 [FINAL STRATEGY] Domain Adaptation (Kaggle + Wiki 50%)")
    
    # 1. 데이터 로드
    print("1️⃣ Loading Source (Kaggle)...")
    kaggle_x, kaggle_y = train_x, train_y
    print("2️⃣ Loading Target (Wiki)...")
    wiki_x, wiki_y = test_x, test_y
    
    if kaggle_x is None or wiki_x is None: return

    # 2. Wiki 데이터 반으로 쪼개기 (섞어서)
    total_wiki = len(wiki_x)
    indices = torch.randperm(total_wiki)
    split_idx = int(total_wiki * 0.5) # 50%만 학습에 사용
    
    wiki_train_x = wiki_x[indices[:split_idx]]
    wiki_train_y = wiki_y[indices[:split_idx]]
    
    wiki_test_x = wiki_x[indices[split_idx:]]
    wiki_test_y = wiki_y[indices[split_idx:]]
    
    print(f"   -> Wiki Split: Train({len(wiki_train_x)}) / Evaluation({len(wiki_test_x)})")

    # 3. 학습 데이터 합치기 (Kaggle + Wiki 절반)
    final_train_x = torch.cat([kaggle_x, wiki_train_x], dim=0)
    final_train_y = torch.cat([kaggle_y, wiki_train_y], dim=0)
    
    print(f"3️⃣ Total Training Set: {len(final_train_x)} images")

    # 4. Dataset & Loader
    train_ds = RAMDataset(final_train_x, final_train_y, EXP_NAME, is_train=True)
    test_ds = RAMDataset(wiki_test_x, wiki_test_y, EXP_NAME, is_train=False) # 오직 Wiki의 나머지 절반으로만 평가
    
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
    # 5. Model (Pretrained)
    model = models.resnet18(weights='IMAGENET1K_V1')
    model.fc = nn.Linear(512, 2)
    model = model.to(DEVICE)
    
    # Fine-tuning이므로 LR을 낮게 설정
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.00005) 
    
    best_acc = 0.0
    print("4️⃣ Start Adaptation Training...")
    
    for epoch in range(EPOCHS):
        model.train()
        for inputs, labels in tqdm(train_loader, desc=f"Ep {epoch+1}", leave=False):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            
            # Mixup 사용
            inputs, ta, tb, lam = mixup_data(inputs, labels, alpha=0.4)
            outputs = model(inputs)
            loss = mixup_criterion(criterion, outputs, ta, tb, lam)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        # 평가
        model.eval()
        preds, acts = [], []
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.to(DEVICE)
                outputs = model(inputs)
                preds.extend(torch.argmax(outputs, 1).cpu().numpy())
                acts.extend(labels.numpy())
        
        acc = accuracy_score(acts, preds)
        print(f"Epoch {epoch+1} | Adapted Test Acc: {acc:.4f}")
        
        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), "model_ram_ADAPTED.pth")
            
    print(f"\n✨ Final Adapted Accuracy: {best_acc:.4f}")
    
    # 혼동 행렬 출력
    cm = confusion_matrix(acts, preds)
    print("-" * 30)
    print("Confusion Matrix:")
    print(cm)
    print("-" * 30)

run_domain_adaptation()


🚀 [FINAL STRATEGY] Domain Adaptation (Kaggle + Wiki 50%)
1️⃣ Loading Source (Kaggle)...
2️⃣ Loading Target (Wiki)...
   -> Wiki Split: Train(5000) / Evaluation(5000)
3️⃣ Total Training Set: 65000 images
4️⃣ Start Adaptation Training...


KeyboardInterrupt: 

In [19]:
def run_domain_adaptation2():
    print("\n🚀 [FINAL STRATEGY] Domain Adaptation (Kaggle + Wiki 50%)")
    
    # 1. 데이터 로드
    print("1️⃣ Loading Source (Kaggle)...")
    kaggle_x, kaggle_y = train_x, train_y
    print("2️⃣ Loading Target (Wiki)...")
    wiki_x, wiki_y = test_x, test_y
    
    if kaggle_x is None or wiki_x is None: return

    # 2. Wiki 데이터 조금만 쪼개기 (섞어서)
    total_wiki = len(wiki_x)
    indices = torch.randperm(total_wiki)
    split_idx = int(total_wiki * 0.1) # 1000개만 학습에 사용
    
    wiki_train_x = wiki_x[indices[:split_idx]]
    wiki_train_y = wiki_y[indices[:split_idx]]
    
    wiki_test_x = wiki_x[indices[split_idx:]]
    wiki_test_y = wiki_y[indices[split_idx:]]
    
    print(f"   -> Wiki Split: Train({len(wiki_train_x)}) / Evaluation({len(wiki_test_x)})")

    # 3. 학습 데이터 합치기 (Kaggle + Wiki 절반)
    final_train_x = wiki_train_x
    final_train_y = wiki_test_y
    
    print(f"3️⃣ Total Training Set: {len(final_train_x)} images")

    # 4. Dataset & Loader
    train_ds = RAMDataset(final_train_x, final_train_y, EXP_NAME, is_train=True)
    test_ds = RAMDataset(wiki_test_x, wiki_test_y, EXP_NAME, is_train=False) # 오직 Wiki의 나머지 절반으로만 평가
    
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
    # 5. Load Models
    model = models.resnet18(pretrained=False) # 구조만 생성
    model.fc = nn.Linear(512, 2)
    
    # 모델 파일 찾기 (경로/이름 유연하게)
    path = f"model_ram_ROBUST.pth"
    model.load_state_dict(torch.load(path, map_location=DEVICE))

    # Fine-tuning이므로 LR을 낮게 설정
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.000001) 
    
    best_acc = 0.0
    print("4️⃣ Start Adaptation Training...")

    model = model.to(DEVICE)
    for epoch in range(EPOCHS):
        model.train()
        for inputs, labels in tqdm(train_loader, desc=f"Ep {epoch+1}", leave=False):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            
            # Mixup 사용
            inputs, ta, tb, lam = mixup_data(inputs, labels, alpha=0.25)
            outputs = model(inputs)
            loss = mixup_criterion(criterion, outputs, ta, tb, lam)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        # 평가
        model.eval()
        preds, acts = [], []
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.to(DEVICE)
                outputs = model(inputs)
                preds.extend(torch.argmax(outputs, 1).cpu().numpy())
                acts.extend(labels.numpy())
        
        acc = accuracy_score(acts, preds)
        print(f"Epoch {epoch+1} | Adapted Test Acc: {acc:.4f}")
        
        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), "model_ram_ADAPTED2.pth")
            
    print(f"\n✨ Final Adapted Accuracy: {best_acc:.4f}")
    
    # 혼동 행렬 출력
    cm = confusion_matrix(acts, preds)
    print("-" * 30)
    print("Confusion Matrix:")
    print(cm)
    print("-" * 30)

run_domain_adaptation2()


🚀 [FINAL STRATEGY] Domain Adaptation (Kaggle + Wiki 50%)
1️⃣ Loading Source (Kaggle)...
2️⃣ Loading Target (Wiki)...
   -> Wiki Split: Train(1000) / Evaluation(9000)
3️⃣ Total Training Set: 1000 images
4️⃣ Start Adaptation Training...


/home/dongwook/workspace/CDFSL/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dongwook/workspace/CDFSL/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1 | Adapted Test Acc: 0.5851


Epoch 2 | Adapted Test Acc: 0.5883


Epoch 3 | Adapted Test Acc: 0.5876


...

Epoch 11 | Adapted Test Acc: 0.5822


Epoch 12 | Adapted Test Acc: 0.5778


Epoch 13 | Adapted Test Acc: 0.5786


Epoch 14 | Adapted Test Acc: 0.5756


Epoch 15 | Adapted Test Acc: 0.5784

✨ Final Adapted Accuracy: 0.5883
------------------------------
Confusion Matrix:
[[2726 1809]
 [1985 2480]]
------------------------------
